In [9]:
from datetime import date
from datetime import datetime
from reportlab.pdfgen import canvas
from reportlab.lib.units import inch
from reportlab.lib.pagesizes import A4
import pandas as pd
import numpy as np
import openpyxl
import requests
import seaborn as sns
import matplotlib.pyplot as plt
from math import pi
import os

In [10]:
class Main:
    def main(self):
        self.url = "https://docs.google.com/spreadsheets/d/1HYjfEe3aCbufbqIXKs0Xz-gfoQNztGhCN1ivx0gZXnc/export?format = xlsx"    
        self.Clone_the_dataset_to_this_machine(self.url)
        self.data = self.creat_data("Student Gradebook.xlsx")
        self.data = self.Add_Month_column(self.data)
        return self.data
                
    def Clone_the_dataset_to_this_machine(self, url):
        a = requests.get(url)
        resp = requests.get(url)
        output = open('Student Gradebook.xlsx',  'wb')
        output.write(resp.content)
        output.close()
        
        
    def creat_data(self, file):
        wb = openpyxl.load_workbook(file) 
        Number_of_sheets = len(wb.sheetnames)
        for i in range(len(wb.sheetnames)):
            try:
                data = pd.concat([data,  pd.read_excel(file, sheet_name = i)])
            except:
                data = pd.read_excel(file, sheet_name = 0)
            #print(data.shape)
        return data

        
    def Add_Month_column(self, data):
        date_list = []
        day_list = []
        year_list = []
        a = 0
        for i in data["Date"]:
            try:
                date_list.append(i.strftime("%B"))
                day_list.append(i.strftime("%d"))
                year_list.append(i.strftime("%Y"))
                a  += 1
            except:
                try:
                    date_list.append(datetime.strptime(str(i), "%d/%m/%Y").strftime("%B"))
                    day_list.append(datetime.strptime(str(i), "%d/%m/%Y").strftime("%d"))
                    year_list.append(datetime.strptime(str(i), "%d/%m/%Y").strftime("%Y"))
                    a += 1
                except:
                    print("Interrupt in a line number : - ", a, "which is having a entry of:-", i)
                    break
        data["Date(only)"] = day_list
        data["Month"] = date_list
        data["Year"] = year_list
        return data

In [11]:
Call = Main()

In [12]:
data = Call.main()

In [13]:
data.head()

,id,Task,Date,Module,Type,Student,Late Submission,Points,Total,Task Winner,Date(only),Month,Year
0,1,Linkedin Profile create/update,2019-07-22 00:00:00,Profile Building,"Intent, Creativity,Content",Chandrima,0,6.0,10,0,22,July,2019
1,1,Linkedin Profile create/update,2019-07-22 00:00:00,Profile Building,"Intent, Creativity,Content",Dipam,0,7.0,10,0,22,July,2019
2,1,Linkedin Profile create/update,2019-07-22 00:00:00,Profile Building,"Intent, Creativity,Content",Durga,0,6.0,10,0,22,July,2019
3,1,Linkedin Profile create/update,2019-07-22 00:00:00,Profile Building,"Intent, Creativity,Content",Kaushal,0,7.0,10,0,22,July,2019
4,1,Linkedin Profile create/update,2019-07-22 00:00:00,Profile Building,"Intent, Creativity,Content",Arya,0,6.0,10,0,22,July,2019


In [14]:
a=data['Task'].value_counts().index
highest=[]
for i in a:
    highest.append(max(data[data['Task']==i]['Points']))

In [15]:
data_highest_table = {'Task':a, 'Highest':highest} 
data_high=pd.DataFrame(data_highest_table)
data=data.merge(data_high, on='Task')

In [16]:
data.head()

,id,Task,Date,Module,Type,Student,Late Submission,Points,Total,Task Winner,Date(only),Month,Year,Highest
0,1,Linkedin Profile create/update,2019-07-22 00:00:00,Profile Building,"Intent, Creativity,Content",Chandrima,0,6.0,10,0,22,July,2019,9.0
1,1,Linkedin Profile create/update,2019-07-22 00:00:00,Profile Building,"Intent, Creativity,Content",Dipam,0,7.0,10,0,22,July,2019,9.0
2,1,Linkedin Profile create/update,2019-07-22 00:00:00,Profile Building,"Intent, Creativity,Content",Durga,0,6.0,10,0,22,July,2019,9.0
3,1,Linkedin Profile create/update,2019-07-22 00:00:00,Profile Building,"Intent, Creativity,Content",Kaushal,0,7.0,10,0,22,July,2019,9.0
4,1,Linkedin Profile create/update,2019-07-22 00:00:00,Profile Building,"Intent, Creativity,Content",Arya,0,6.0,10,0,22,July,2019,9.0


In [25]:
name = "Shakib"
month = "July"

In [26]:
def rank_of_the_student(name, month):
    data_working=data[data['Month']==month]
    data_task=data_working[data_working['Module']!='Ritual']
    data_rank=data_task.groupby('Student')['Points'].sum().sort_values(ascending=False).reset_index()
    inx=data_rank['Student'].tolist()
    i=pd.Index(inx).get_loc(name)
    return str(i+1)
rank_of_the_student(name, month)

'14'

In [27]:
def late_Submition_Ratio(name, month):
    data_working=data[data['Month']==month]
    data_student=data_working[data_working['Student']==name]
    p=len(data_working[data_working['Module']!='Ritual']['Task'].value_counts().index)
    q=len(data_student[data_student['Late Submission']==1])
    ratio=q/p
    return str(ratio)

late_Submition_Ratio(name, month)

'0.0'

In [28]:
def percentage_of_the_student(name, month):
    data_working=data[data['Month']==month]
    data_student=data_working[data_working['Student']==name]
    data_task=data_student[data_student['id']!=0]
    a=data_task['Points'].sum()
    b=data_task['Total'].sum()
    percentage=(a/b)*100
    return str(percentage)

percentage_of_the_student(name, month)

'73.33333333333333'

In [29]:
def percentile_of_the_student(name, month):
    data_working=data[data['Month']==month]
    a=int(rank_of_the_student(name, month))
    b=len(data['Student'].value_counts().index)
    c=((b-a)/b)*100
    return str(c)

percentage_of_the_student(name, month)

'73.33333333333333'

In [30]:
def table_Content(name, month):
    data_working=data[data['Month']==month]
    data_student=data_working[data_working['Student']==name]
    x1=data_student.pivot_table(index='Module',values='Points',aggfunc='sum')
    y1=data_student.pivot_table(index='Module',values='Total',aggfunc='sum')
    w1=data_student.pivot_table(index='Module',values='Highest',aggfunc='sum')
    z1=pd.concat([x1,y1,w1],axis=1)
    subject_percentile=[x1['Points']/y1['Total']*100]
    z1['Percentile']=" "
    for i in range(len(z1)):
        z1['Percentile'][i]=subject_percentile[0][i]
    z1.to_html('z_total.html')
    df=pd.read_html('z_total.html')
    Table_elements=df[0].values.tolist()
    return Table_elements

table_Content(name, month)

C:\Users\Shakib\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


[['Introduction to Data Science', 37.5, 50, 43.0, 75],
 ['Profile Building', 17.5, 25, 22.5, 70],
 ['Ritual', 14.0, 14, 14.0, 100]]

In [31]:
def table_summary(name, month):
    data_working=data[data['Month']==month]
    data_student=data_working[data_working['Student']==name]
    x1=data_student.pivot_table(index='Module',values='Points',aggfunc='sum')
    y1=data_student.pivot_table(index='Module',values='Total',aggfunc='sum')
    w1=data_student.pivot_table(index='Module',values='Highest',aggfunc='sum')
    z1=pd.concat([x1,y1,w1],axis=1)
    subject_percentile=[x1['Points']/y1['Total']*100]
    z1['Percentile']=" "
    for i in range(len(z1)):
        z1['Percentile'][i]=subject_percentile[0][i]
    j=z1['Points'].sum()
    k=z1['Total'].sum()
    l=z1['Highest'].sum()
    t=list([j,k,l])
    return t

table_summary(name, month)

C:\Users\Shakib\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


[69.0, 89, 79.5]

In [32]:
def task_win(name, month):
    data_working=data[data['Month']==month]
    data_student=data_working[data_working['Student']==name]
    m=len(data_student[data_student['Task Winner']==1])
    return m

task_win(name, month)

0

In [35]:
data["Student"].value_counts().index

Index(['Dipam', 'Surabhi', 'Chandrima', 'Ujjainee', 'Shakib', 'Kunal',
       'Purbita', 'Sonali', 'Vishal', 'Siddhishikha', 'Durga', 'Sharika',
       'Apurwa', 'Prasoon', 'Kaushal', 'Swaastick', 'Shantanu', 'Bhavna',
       'Roumyak', 'Arya', 'Anjali', 'Swaastik', 'Wrick', 'Nitish', 'Sudhanshu',
       'Sushree'],
      dtype='object')

In [2]:
""" 'Swaastik', 'Wrick', 'Nitish', 'Sudhanshu','Sushree']"""

" 'Swaastik', 'Wrick', 'Nitish', 'Sudhanshu','Sushree']"